In [17]:
import tensorflow as tf

model = tf.keras.Sequential([
    # Input: 128x128 works better for mobile than 256x256
    tf.keras.layers.InputLayer(input_shape=(128, 128, 3)),

    # Mobile-optimized layers
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),  # Tiny filters
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),  # Lightweight
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Reduces overfitting
    tf.keras.layers.Dense(17, activation='softmax')
])
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [18]:
def preprocess(image, label):
    # Convert to float32 and normalize to [-1, 1]
    image = tf.cast(image, tf.float32) / 127.5 - 1.0
    # Glove-specific augmentations
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image, label

# Load dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    "C:/Users/crese/Downloads/caddy-gestures-complete-v2-release-all-scenarios-fast.ai",
    image_size=(128, 128),
    batch_size=32
).map(preprocess)  # Apply augmentations here

# Manual validation split
val_split = 0.2
train_samples = int((1 - val_split) * len(train_ds))
val_ds = train_ds.skip(train_samples)
train_ds = train_ds.take(train_samples)

Found 18482 files belonging to 17 classes.


In [20]:
# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5),
    tf.keras.callbacks.ModelCheckpoint('glove_gestures.keras')
]

# Train
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,  # Use manually split validation
    callbacks=callbacks
)

Epoch 1/10
462/462 ━━━━━━━━━━━━━━━━━━━━ 172s 368ms/step - accuracy: 0.2219 - loss: 2.6236 - val_accuracy: 0.4346 - val_loss: 1.7526
Epoch 2/10
462/462 ━━━━━━━━━━━━━━━━━━━━ 170s 368ms/step - accuracy: 0.4051 - loss: 1.8594 - val_accuracy: 0.5395 - val_loss: 1.4485
Epoch 3/10
462/462 ━━━━━━━━━━━━━━━━━━━━ 201s 435ms/step - accuracy: 0.4997 - loss: 1.5389 - val_accuracy: 0.6160 - val_loss: 1.1795
Epoch 4/10
462/462 ━━━━━━━━━━━━━━━━━━━━ 199s 432ms/step - accuracy: 0.5797 - loss: 1.2481 - val_accuracy: 0.6628 - val_loss: 1.0403
Epoch 5/10
462/462 ━━━━━━━━━━━━━━━━━━━━ 219s 473ms/step - accuracy: 0.6482 - loss: 1.0263 - val_accuracy: 0.7001 - val_loss: 0.9413
Epoch 6/10
462/462 ━━━━━━━━━━━━━━━━━━━━ 238s 515ms/step - accuracy: 0.7059 - loss: 0.8589 - val_accuracy: 0.7271 - val_loss: 0.8854
Epoch 7/10
462/462 ━━━━━━━━━━━━━━━━━━━━ 261s 565ms/step - accuracy: 0.7513 - loss: 0.7160 - val_accuracy: 0.7534 - val_loss: 0.8159
Epoch 8/10
462/462 ━━━━━━━━━━━━━━━━━━━━ 253s 547ms/step - accuracy: 0.7908 -